# [Лабораторная работа #7](https://brestprog.by/topics/skeleton/)

Есть N узлов, которые необходимо объединить в сеть. Известна стоимость прокладки  оптоволоконного кабеля между любой парой узлов. Требуется спроектировать связную сеть (сеть, между любыми узлами которой можно передать сигнал) минимальной стоимости.  Задачу реализовать 2-мя алгоритмами.


In [39]:
from typing import List, Tuple
from collections import namedtuple

In [42]:
class Graph:
    weights: List[List[float]]
        
    def __init__(self, vertices_num):
        self.weights = [[float('inf') for _ in range(vertices_num)] for __ in range(vertices_num)]
        
    def get_num_vertices(self) -> int:
        return len(self.weights)

    def set_weight_between_vertices(self, vertex_1, vertex_2, weight) -> None:
        self.weights[vertex_1-1][vertex_2-1] = weight
        self.weights[vertex_2-1][vertex_1-1] = weight
    
    def get_weight_between_vertices(self, vertex_index_1: int, vertex_index_2: int) -> float:
        return self.weights[vertex_index_1][vertex_index_2]
    
    def get_neighborhood(self, vertex_index: int):
        return [
            (adjacent_vertex_index, weight) 
                for adjacent_vertex_index, weight in enumerate(self.weights[vertex_index])
                    if weight != float("inf")
        ]


# Prim algorithm

In [43]:
def find_start_vertex_for_prim(graph: Graph):
    """
    this function finds vertex wich is incident to an edge with min weight
    """
    min_vertex_index: int = None
    min_weight: int = float('inf')
    
    for adjacent_vertex_index, adjacent_weights in enumerate(graph.weights):
        if min(adjacent_weights) < min_weight:
            min_vertex_index, min_weight = adjacent_vertex_index, min(adjacent_weights)
    
    return min_vertex_index


In [59]:
def run_prim(graph: Graph):
    """
    this function runs prim algorithm to the @param: graph
    function returns edges of the @param: graph skeleton and weight of this skeleton
    """
    skeleton_vertices: List[Tuple[int, int]] = []
    skeleton_edges: List[Tuple[int, int]] = []
    skeleton_weight: int = 0
    que: List[Tuple[int, int], Tuple[int, int]] = []
    # que: (vertex number, distance between vertex and skeleton) 
    #       and (edge between vertex and skeleton)
    start_vertex_index: int = find_start_vertex_for_prim(graph)
    que.append(((start_vertex_index, 0), (start_vertex_index, start_vertex_index)))
    
    while que:
        # min distance to skeleton
        current_pair, current_edge = min(que, key=lambda item: item[0][1])
        que.remove(min(que, key=lambda item: item[0][1]))

        vertex, distance_to_skeleton = current_pair
        
        if vertex in skeleton_vertices:
            continue
            
        skeleton_vertices.append(vertex)
        skeleton_edges.append(current_edge)
        skeleton_weight += distance_to_skeleton
        
        for adjacent_pair in graph.get_neighborhood(vertex):
            adjacent_vertex, adjacent_distance_to_skeleton = adjacent_pair
            
            if adjacent_vertex not in skeleton_vertices:
                que.append(((adjacent_vertex, adjacent_distance_to_skeleton), (vertex, adjacent_vertex)))

        
    return skeleton_edges[1:], skeleton_weight
        

In [60]:
graph = Graph(10)
graph.set_weight_between_vertices(1, 2, 2)
graph.set_weight_between_vertices(1, 3, 3)
graph.set_weight_between_vertices(1, 4, 4)
graph.set_weight_between_vertices(2, 5, 4)
graph.set_weight_between_vertices(2, 6, 5)
graph.set_weight_between_vertices(2, 3, 3)
graph.set_weight_between_vertices(3, 5, 1)
graph.set_weight_between_vertices(3, 6, 6)
graph.set_weight_between_vertices(3, 4, 1)
graph.set_weight_between_vertices(4, 6, 5)
graph.set_weight_between_vertices(4, 10, 3)
graph.set_weight_between_vertices(5, 9, 7)
graph.set_weight_between_vertices(5, 6, 2)
graph.set_weight_between_vertices(6, 8, 6)
graph.set_weight_between_vertices(6, 9, 1)
graph.set_weight_between_vertices(6, 10, 4)
graph.set_weight_between_vertices(6, 7, 2)
graph.set_weight_between_vertices(7, 9, 8)
graph.set_weight_between_vertices(7, 10, 1)
graph.set_weight_between_vertices(8, 9, 5)
graph.set_weight_between_vertices(8, 10, 3)
graph.set_weight_between_vertices(9, 10, 4)

In [61]:
skeleton_edges, skeleton_weight = run_prim(graph)

In [62]:
skeleton_edges

[(2, 3), (2, 4), (4, 5), (5, 8), (5, 6), (6, 9), (2, 0), (0, 1), (9, 7)]

In [63]:
skeleton_weight

16

# Kruskal algorithm

In [64]:
first_connectivity_component = [None for _ in range(10)]
second_connectivity_component = [None for _ in range(10)]

In [65]:
def init_components(first_component, second_component) -> None:
    for i in range(10):
        first_component[i] = i
        second_component[i] = 0


In [66]:
def get_root(v: int, first_component: list) -> int:
    if first_component[v] == v:
        return v
    else:
        first_component[v] = get_root(first_component[v], first_component)
        return first_component[v]  # На выходе из рекурсии переподвешиваем v

In [67]:
def merge(a: int, b: int, first_component: list, second_component: list) -> bool:
    firt_root: int = get_root(a, first_component)
    second_root: int = get_root(b, first_component)

    if firt_root == second_root:
        return False
    else:
        if second_connectivity_component[firt_root] < second_component[second_root]:
            first_component[firt_root] = second_root
        elif second_connectivity_component[second_root] < second_component[firt_root]:
            first_component[second_root] = firt_root
        else:
            first_component[firt_root] = second_root
            second_component[second_root] += 1
        

        return True

In [68]:
import functools

In [84]:
@functools.total_ordering
class Edge:
    
    def __init__(self, a, b, length=None):
        self.a = a-1
        self.b = b-1
        self.length = length
        
    def __lt__(self, other):
        return self.length < other.length
    
    def __eq__(self, other):
        return self.length == other.length
    
    def __str__(self):
        return f'<a:{self.a+1} b:{self.b+1}>'
    
    def __repr__(self):
        return f'<a:{self.a+1} b:{self.b+1}>'

In [70]:
edges = []
edges.append(Edge(1, 2, 2))
edges.append(Edge(1, 3, 3))
edges.append(Edge(1, 4, 4))
edges.append(Edge(2, 5, 4))
edges.append(Edge(2, 6, 5))
edges.append(Edge(2, 3, 3))
edges.append(Edge(3, 5, 1))
edges.append(Edge(3, 6, 6))
edges.append(Edge(3, 4, 1))
edges.append(Edge(4, 6, 5))
edges.append(Edge(4, 10, 3))
edges.append(Edge(5, 9, 7))
edges.append(Edge(5, 6, 2))
edges.append(Edge(6, 8, 6))
edges.append(Edge(6, 9, 1))
edges.append(Edge(6, 10, 4))
edges.append(Edge(6, 7, 2))
edges.append(Edge(7, 9, 8))
edges.append(Edge(7, 10, 1))
edges.append(Edge(8, 9, 5))
edges.append(Edge(8, 10, 3))
edges.append(Edge(9, 10, 4))

In [75]:
def run_kruscal(edges: List[Edge], first_component: list, second_component: list):
    skeleton_weight: int = 0
    skeleton: list = []

    edges.sort()
    init_components(first_component, second_component)
    for edge in edges:
        if merge(edge.a, edge.b, first_component, second_component):
            skeleton.append(edge)
            skeleton_weight += edge.length
    return skeleton

In [81]:
run_kruscal(edges, first_connectivity_component, second_connectivity_component)

[<a:3 b:5>,
 <a:3 b:4>,
 <a:6 b:9>,
 <a:7 b:10>,
 <a:1 b:2>,
 <a:5 b:6>,
 <a:6 b:7>,
 <a:1 b:3>,
 <a:8 b:10>]

# Тесты

In [34]:
import unittest

In [86]:
class TestGraph(unittest.TestCase):

    def setUp(self):
        self.graph_a = Graph(10)
        self.graph_a.set_weight_between_vertices(1, 2, 2)
        self.graph_a.set_weight_between_vertices(1, 3, 3)
        self.graph_a.set_weight_between_vertices(1, 4, 4)
        self.graph_a.set_weight_between_vertices(2, 5, 4)
        self.graph_a.set_weight_between_vertices(2, 6, 5)
        self.graph_a.set_weight_between_vertices(2, 3, 3)
        self.graph_a.set_weight_between_vertices(3, 5, 1)
        self.graph_a.set_weight_between_vertices(3, 6, 6)
        self.graph_a.set_weight_between_vertices(3, 4, 1)
        self.graph_a.set_weight_between_vertices(4, 6, 5)
        self.graph_a.set_weight_between_vertices(4, 10, 3)
        self.graph_a.set_weight_between_vertices(5, 9, 7)
        self.graph_a.set_weight_between_vertices(5, 6, 2)
        self.graph_a.set_weight_between_vertices(6, 8, 6)
        self.graph_a.set_weight_between_vertices(6, 9, 1)
        self.graph_a.set_weight_between_vertices(6, 10, 4)
        self.graph_a.set_weight_between_vertices(6, 7, 2)
        self.graph_a.set_weight_between_vertices(7, 9, 8)
        self.graph_a.set_weight_between_vertices(7, 10, 1)
        self.graph_a.set_weight_between_vertices(8, 9, 5)
        self.graph_a.set_weight_between_vertices(8, 10, 3)
        self.graph_a.set_weight_between_vertices(9, 10, 4)
        
        self.edges = []
        self.edges.append(Edge(1, 2, 2))
        self.edges.append(Edge(1, 3, 3))
        self.edges.append(Edge(1, 4, 4))
        self.edges.append(Edge(2, 5, 4))
        self.edges.append(Edge(2, 6, 5))
        self.edges.append(Edge(2, 3, 3))
        self.edges.append(Edge(3, 5, 1))
        self.edges.append(Edge(3, 6, 6))
        self.edges.append(Edge(3, 4, 1))
        self.edges.append(Edge(4, 6, 5))
        self.edges.append(Edge(4, 10, 3))
        self.edges.append(Edge(5, 9, 7))
        self.edges.append(Edge(5, 6, 2))
        self.edges.append(Edge(6, 8, 6))
        self.edges.append(Edge(6, 9, 1))
        self.edges.append(Edge(6, 10, 4))
        self.edges.append(Edge(6, 7, 2))
        self.edges.append(Edge(7, 9, 8))
        self.edges.append(Edge(7, 10, 1))
        self.edges.append(Edge(8, 9, 5))
        self.edges.append(Edge(8, 10, 3))
        self.edges.append(Edge(9, 10, 4))
        
        self.first_connectivity_component = [None for _ in range(10)]
        self.second_connectivity_component = [None for _ in range(10)]
        
        
    def test_run_prim(self):
        self.assertEqual(
            run_prim(self.graph_a),
            (
                [(2, 3), (2, 4), (4, 5), (5, 8), (5, 6), (6, 9), (2, 0), (0, 1), (9, 7)],
                16
            )
        )
        
    def test_run_cruscal(self):
        self.assertEqual(
            run_kruscal(self.edges, self.first_connectivity_component, self.second_connectivity_component),
            [
                Edge(3, 5, 1),
                Edge(3, 4, 1),
                Edge(6, 9, 1),
                Edge(7, 10, 1),
                Edge(1, 2, 2),
                Edge(5, 6, 2),
                Edge(6, 7, 2),
                Edge(1, 3, 3),
                Edge(8, 10, 3)
            ]
        )
        

unittest.main(argv=[''], verbosity=2, exit=False)


test_run_cruscal (__main__.TestGraph) ... ok
test_run_prim (__main__.TestGraph) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.011s

OK
